In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from datetime import datetime
from math import ceil

pd.set_option('display.max_columns', None)

In [2]:
art = pd.read_csv('clients_art_wo_names.csv')
# Я предварительно поменял в исходном файле десятичный разделитель с запятой на точку
print(art[:5])
print('Shape')
print(art.shape)
print('Unique')
for column in art.columns:
    print("{:8s}\t".format(column), art[column].unique().shape[0])
# Считаем, что артикул art_id - просто случайное число, а из кодов групп в дереве товаров
# можно получить какую-то метрику. Я предполагаю, что вода, хлеб и крупа не окажутся 
# одновременно в одной группе товаров третьего уровня, а условные вода "Шишкин лес" и "Святой
# источник окажутся в одной группе lvl2_id, а lvl3_id у них будет отличаться. Задам вручную вес
# для каждого уровня
NN = 20 # Проверено, коллизий нет
art['art'] = NN**3 * art['lvl0_id'] + NN**2 * art['lvl1_id'] + NN * art['lvl2_id'] + art['lvl3_id']
print('Shape')
print(art.shape)
print('Unique')
for column in art.columns:
    print("{:8s}\t".format(column), art[column].unique().shape[0])
print(art[:5])
#for i in range(4):
#    print(art['lvl{}_id'.format(i)].max())

     art_id  weight  lvl0_id  lvl1_id  lvl2_id  lvl3_id
0    192584    0.21    16752     1677     7263    15232
1    310536    1.00    16352       34    16258      574
2  15321862    0.25    16722      655    16094    16092
3    114853    0.60     2135     1953     1962     7961
4  16522966    0.50      679      912      970    14862
Shape
(12382, 6)
Unique
art_id  	 12382
weight  	 486
lvl0_id 	 35
lvl1_id 	 185
lvl2_id 	 693
lvl3_id 	 1511
Shape
(12382, 7)
Unique
art_id  	 12382
weight  	 486
lvl0_id 	 35
lvl1_id 	 185
lvl2_id 	 693
lvl3_id 	 1511
art     	 1511
     art_id  weight  lvl0_id  lvl1_id  lvl2_id  lvl3_id        art
0    192584    0.21    16752     1677     7263    15232  134847292
1    310536    1.00    16352       34    16258      574  131155334
2  15321862    0.25    16722      655    16094    16092  134375972
3    114853    0.60     2135     1953     1962     7961   17908401
4  16522966    0.50      679      912      970    14862    5831062


In [3]:
train = pd.read_csv('clients_train.csv')
test = pd.read_csv('clients_test_cut.csv')
print('Train')
print(train[:3])
print('Test')
print(test[:3])
print('----------------------------------------')
print('Unique values')
print('Column\t\tTrain\t\tTest')
print('Shape\t\t', train.shape, '\t', test.shape)
for column in test.columns:
    print("{:8s}\t".format(column), train[column].unique().shape[0], '\t\t', test[column].unique().shape[0])
print("\n{:8s}\t".format('client_id'), train['client_id'].unique().shape[0])
print('----------------------------------------')
# Посмотрим, есть ли пересечения в данных
print('Test in train')
for column in test.columns:
    counts = test[column].isin(train[column]).value_counts()
    print("{:8s}\t\t".format(column), counts[counts.index == True].values)

Train
       day_id    art_id  whs_id       txn_id  sale_qnty  reg_opsum  \
0  2018-02-04  16550635   26517  14914458657      1.000     85.900   
1  2018-02-03    243028   26517  14905137106      1.000    149.900   
2  2018-02-03    327275   26517  14895612196      0.173     60.533   

   fact_opsum  mission_id  client_id  
0      85.900          -1  116871314  
1     149.900          -1  126493766  
2      60.533          -1  105675535  
Test
       day_id    art_id  whs_id       txn_id  sale_qnty  reg_opsum  \
0  2018-04-01    275845   26517  15578368583        1.0       24.9   
1  2018-04-01  16751680   26517  15578368583        1.0       29.9   
2  2018-04-01  15333128   26517  15578368583        1.0       19.9   

   fact_opsum  mission_id  
0        24.9         220  
1        29.9         220  
2        19.9         220  
----------------------------------------
Unique values
Column		Train		Test
Shape		 (8093, 9) 	 (3581, 8)
day_id  	 59 		 30
art_id  	 2151 		 1304
whs_id  	 1 

In [4]:
train = pd.read_csv('clients_train.csv')
test = pd.read_csv('clients_test_cut.csv')

def gen_features(X):
    X = X.copy()
    X = X.join(art.set_index('art_id'), on = 'art_id')
    # Артикул можно бы и удалить, но градиентный бустинг достаточно устойчив
    # X.drop(['art_id', 'lvl0_id', 'lvl1_id', 'lvl2_id', 'lvl3_id'], inplace = True, axis = 1)
    
    # Одно число в столбце. Для обучения модели оно не поможет
    X.drop('whs_id', inplace = True, axis = 1)
    # Так как и количество, и вес могут быть дробными, введем еще одну переменную
    X['qty'] = X['sale_qnty'].multiply(X['weight'])
    # X.drop(['sale_qnty', 'weight'], inplace = True, axis = 1)
    X['prod_disc'] = (X['reg_opsum'] - X['fact_opsum']).divide(X['reg_opsum'])
    X['day_id'] = X['day_id'].map(lambda x: datetime.strptime(x, "%Y-%m-%d"))
    X['dayofweek'] = X['day_id'].map(lambda x: x.dayofweek)
    X['day_id'] = X['day_id'].map(lambda x: x.timestamp())
    return X

X_train, X_test = gen_features(train), gen_features(test)
# Fill NaN in 'prod_disc'
X_train.fillna(value = 1, inplace = True)
X_test.fillna(value = 1, inplace = True)
#my_train = X_train.copy()

#y_train = X_train['client_id']
#X_train.drop('client_id', inplace = True, axis = 1)
print('Test in train')
for column in X_test.columns:
    counts = X_test[column].isin(X_train[column]).value_counts()
    print("{:8s}\t\t".format(column), counts[counts.index == True].values)
print('----------------------------------------')
print('Unique values')
print('Column\t\tTrain\t\tTest')
print('Shape\t\t', X_train.shape, '\t', X_test.shape)
for column in X_test.columns:
    print("{:8s}\t".format(column), X_train[column].unique().shape[0], '\t\t', X_test[column].unique().shape[0])
print('----------------------------------------')

Test in train
day_id  		 []
art_id  		 [2977]
txn_id  		 []
sale_qnty		 [3387]
reg_opsum		 [2803]
fact_opsum		 [2767]
mission_id		 [3576]
weight  		 [3551]
lvl0_id 		 [3581]
lvl1_id 		 [3571]
lvl2_id 		 [3551]
lvl3_id 		 [3502]
art     		 [3502]
qty     		 [3399]
prod_disc		 [2849]
dayofweek		 [3581]
----------------------------------------
Unique values
Column		Train		Test
Shape		 (8093, 17) 	 (3581, 16)
day_id  	 59 		 30
art_id  	 2151 		 1304
txn_id  	 2043 		 950
sale_qnty	 678 		 375
reg_opsum	 2299 		 1063
fact_opsum	 2591 		 1163
mission_id	 20 		 20
weight  	 211 		 175
lvl0_id 	 34 		 33
lvl1_id 	 141 		 127
lvl2_id 	 403 		 312
lvl3_id 	 667 		 470
art     	 667 		 470
qty     	 820 		 508
prod_disc	 1618 		 665
dayofweek	 7 		 7
----------------------------------------


In [5]:
def mk_new_data(X_train, X_test):
    # Стоило написать дополнительную функцию для формирования train и test. Было бы проще и более читаемо
    X_train = X_train.copy()
    X_test = X_test.copy()
    # Округляем количество покупок вверх до целого, чтобы потом посчитать количество товаров в чеке
    X_train['qty_int'] = X_train['sale_qnty'].map(ceil) 
    X_test['qty_int'] = X_test['sale_qnty'].map(ceil) 
    # Просуммируем количество товаров, определим общую сумму чека
    train = X_train.groupby('txn_id')['qty', 'reg_opsum', 'fact_opsum', 'qty_int'].sum()
    test = X_test.groupby('txn_id')['qty', 'reg_opsum', 'fact_opsum', 'qty_int'].sum()
    # Добавим дату и день недели к данным
    train = train.join(X_train.set_index('txn_id')[['day_id', 'dayofweek', 'mission_id']])
    test = test.join(X_train.set_index('txn_id')[['day_id', 'dayofweek', 'mission_id']])
    # Посчитаем скидку по чеку
    train['prod_disc'] = (train['reg_opsum'] - train['fact_opsum']).divide(train['reg_opsum'])
    test['prod_disc'] = (test['reg_opsum'] - test['fact_opsum']).divide(test['reg_opsum'])
    # Определим среднюю стоимость товара в чеке
    train['avg'] = train['reg_opsum'].divide(train['qty_int'])
    test['avg'] = test['reg_opsum'].divide(test['qty_int'])

    y_train = X_train[['txn_id', 'client_id']].drop_duplicates().set_index('txn_id')
    
       
    ohe = OneHotEncoder(sparse = False)
    temp = pd.concat([X_train[['lvl0_id']], X_test[['lvl0_id']]])['lvl0_id'].map(str)
    ohe.fit(temp.values.reshape(-1, 1))
    ohe_train = pd.DataFrame(ohe.transform(X_train['lvl0_id'].values.reshape(-1, 1)), index = X_train.index).join(X_train['txn_id'])
    ohe_test = pd.DataFrame(ohe.transform(X_test['lvl0_id'].values.reshape(-1, 1)), index = X_test.index).join(X_test['txn_id'])
    ohe_train = ohe_train.groupby('txn_id').sum()
    ohe_test = ohe_test.groupby('txn_id').sum()
    ohe_train.columns = ['lvl0_' + str(int(i)) for i in range(ohe_train.shape[1])]
    ohe_test.columns = ['lvl0_' + str(int(i)) for i in range(ohe_test.shape[1])]
    #train = train.join(ohe_train)
    #test = test.join(ohe_test)
    
    ohe = OneHotEncoder(sparse = False)
    temp = pd.concat([X_train[['lvl1_id']], X_test[['lvl1_id']]])['lvl1_id'].map(str)
    ohe.fit(temp.values.reshape(-1, 1))
    ohe_train = pd.DataFrame(ohe.transform(X_train['lvl1_id'].values.reshape(-1, 1)), index = X_train.index).join(X_train['txn_id'])
    ohe_test = pd.DataFrame(ohe.transform(X_test['lvl1_id'].values.reshape(-1, 1)), index = X_test.index).join(X_test['txn_id'])
    ohe_train = ohe_train.groupby('txn_id').sum()
    ohe_test = ohe_test.groupby('txn_id').sum()
    ohe_train.columns = ['lvl1_' + str(int(i)) for i in range(ohe_train.shape[1])]
    ohe_test.columns = ['lvl1_' + str(int(i)) for i in range(ohe_test.shape[1])]
    #train = train.join(ohe_train)
    #test = test.join(ohe_test)
    
    ohe = OneHotEncoder(sparse = False)
    temp = pd.concat([X_train[['lvl2_id']], X_test[['lvl2_id']]])['lvl2_id'].map(str)
    ohe.fit(temp.values.reshape(-1, 1))
    ohe_train = pd.DataFrame(ohe.transform(X_train['lvl2_id'].values.reshape(-1, 1)), index = X_train.index).join(X_train['txn_id'])
    ohe_test = pd.DataFrame(ohe.transform(X_test['lvl2_id'].values.reshape(-1, 1)), index = X_test.index).join(X_test['txn_id'])
    ohe_train = ohe_train.groupby('txn_id').sum()
    ohe_test = ohe_test.groupby('txn_id').sum()
    ohe_train.columns = ['lvl2_' + str(int(i)) for i in range(ohe_train.shape[1])]
    ohe_test.columns = ['lvl2_' + str(int(i)) for i in range(ohe_test.shape[1])]
    #train = train.join(ohe_train)
    #test = test.join(ohe_test)
    
    # Удалим все дубликаты
    train = train.reset_index().drop_duplicates()
    train = train.set_index('txn_id')
    test = test.reset_index().drop_duplicates()
    test = test.set_index('txn_id')
    train = train.sort_index()
    test = test.sort_index()
    train.fillna(value = 1, inplace = True)
    test.fillna(value = 1, inplace = True)
    return train, y_train, test

train, y_train, test = mk_new_data(X_train, X_test)


In [ ]:
import pylab
colors = [int(i % y_train['client_id'].unique().shape[0]) for i in y_train.reset_index(drop = True).index]
for i in train.columns:
    for j in train.columns:
        pylab.scatter(train[i], train[j], c=colors)
        pylab.xlabel(i)
        pylab.ylabel(i)
        pylab.show()